In [1]:
import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd

## LSTMCell

* seq_length : 332일치의 데이터가 있을 때 seq_length가 10이라면 10일치의 데이터가
하나의 시퀀스임. 그리고 시퀀스 개수는 332-10+1=323개.

* batch_size : 여러 개의 시퀀스를 한 번에 처리하기 위한 묶음. 332일치의 데이터가 있을 때 batch_size가 32라면 10개의 시퀀스가 32개(연속된 320일 치의 데이터가 있다는 게 아님). 그러므로 이 경우 배치 하나의 shape은 (32,10,2).

* hidden_size : hidden_state의 dimension(shape OR feature 개수). hidden_state의 dimension은 input_data의 dimension(feature)과 독립. hidden_state는 과거의 모든 입력 정보와 LSTM의 내부 계산을 반영하여 다음 타임스텝으로 전달하는 역할이기 때문. 즉, LSTM이 input_data를 처리한 결과로 생성된 학습 표현(learned representation).

---

> hidden_state의 shape은 (num_layers, batch_size, hidden_size), input_data의 shape은 (batch_size, seq_length, feature_size).
cell_state의 shape은 (num_layers, batch_size, hidden_size), output의 shape은 (batch_size, seq_length, hidden_size). 이때 각 LSTM Cell에 들어가는
Xt의 shape은 (1,1,feature_size).
=> 가중치 행렬은 이의 shape을 맞춰줘야 함.

> nn.Module은 파이토치에서 모든 nueral network 구성에 필수. 미분과 매개변수 관리 등을 자동으로 해줌

> nn.Module을 상속하게 되면 기본적으로 __init__()과 foward()를 override하게 됨

> reset_parameter 메서드가 필요한 이유는, 파라미터(=가중치) 학습(즉, 적절한 파라미터 값을 찾아나가는 과정) 시에 기울기 소실문제가 발생할 수 있는데 이를 방지하고, 과대적합/과소적합 시에 학습이 제대로 되지 않는 상황을 막기 위해.

In [2]:
class LSTMCell(nn.Module):
  # 모델의 변수를 초기화하는 매서드
  def __init__(self, input_size, hidden_size, bias=True):
    super(LSTMCell, self).__init__()
    # 각 Cell에 들어올 input data feature(차원, dimension) 개수(크기)
    self.input_size = input_size
    # hidden state의 feature(차원, dimension) 개수(크기)
    self.hidden_size = hidden_size
    # 편향
    self.bias = bias

    # 학습에 필요한 input gate 파라미터들을 생성
    self.Wx_i = nn.Parameter(torch.Tensor(input_size, hidden_size)) # x에 대한 가중치 탠서
    self.Wh_i = nn.Parameter(torch.Tensor(hidden_size, hidden_size)) # h에 대한 가중치 탠서
    self.b_i = nn.Parameter(torch.Tensor(hidden_size)) if bias else None
    # 학습에 필요한 forget gate 파라미터들을 생성
    self.Wx_f = nn.Parameter(torch.Tensor(input_size, hidden_size))
    self.Wh_f = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
    self.b_f = nn.Parameter(torch.Tensor(hidden_size)) if bias else None
    # 학습에 필요한 input gate 파라미터들을 생성
    self.Wx_o = nn.Parameter(torch.Tensor(input_size, hidden_size))
    self.Wh_o = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
    self.b_o = nn.Parameter(torch.Tensor(hidden_size)) if bias else None
    # 학습에 필요한 예비 cell_state 파라미터들을 생성
    self.Wx_c = nn.Parameter(torch.Tensor(input_size, hidden_size))
    self.Wh_c = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
    self.b_c = nn.Parameter(torch.Tensor(hidden_size)) if bias else None

    def reset_parameters(self):
      # 가중치 초기화 (Xavier 초기화 등)
      stdv = 1.0 / (self.hidden_size ** 0.5)
      for weight in self.parameters():
        nn.init.uniform_(weight, -stdv, stdv)

  # input data에 모델이 적용할 연산을 구현하는 매서드
  def forward(self,x,hidden):
    """
    x의 shape : (batch_size, input_size)
    hidden가 반환하는 값 : (h_prev, c_prev)
    """
    # hidden 변수에 이전 시점의 hidden_state, cell_state가 들어있음. set형태로.
    h_prev, c_prev = hidden
    # Input gate
    i_t = torch.sigmoid(x @ self.Wx_i + h_prev @ self.Wh_i + (self.b_i if self.bias else 0))
    # Forget gate
    f_t = torch.sigmoid(x @ self.Wx_f + h_prev @ self.Wh_f + (self.b_f if self.bias else 0))
    # 예비 cell_state값
    c_tilde = torch.tanh(x @ self.Wx_c + h_prev @ self.Wh_c + (self.b_c if self.bias else 0))
    # Output gate
    o_t = torch.sigmoid(x @ self.Wx_o + h_prev @ self.Wh_o + (self.b_o if self.bias else 0))
    # Cell state
    c_t = f_t * c_prev + i_t * c_tilde
    # hidden state
    h_t = o_t * torch.tanh(c_t)

    return h_t, c_t

##LSTM

> Cell로 구성된 레이어들을 만들고, 이전 레이어에서 넘어온 값을 다음 레이어가 받을 수 있게 모델을 구성. 아래 코드에서 cell은 해당 레이어를 구성하는 여러 개의 LSTMCell을 의미. 이때 각 레이어에는 LSTMCell 객체가 들어가는데, 해당 객체는 레이어 별로 한 개씩만 들어가는데 하나의 LSTMCell 객체는 input_size에 따라 여러 개의 cell을 가질 수 있음.

In [9]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers=1, bias=True):
    super(LSTM, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    # LSTMCell들로 구성된 레이어들을 생성
    self.layers = nn.ModuleList()
    for layer in range(num_layers):
      current_input_size = input_size if layer == 0 else hidden_size
      self.layers.append(LSTMCell(current_input_size, hidden_size, bias))

  # 다음 레이어로 forward 하는 거임. LSTMCell에선 다음 Cell로 foward 하는 거고.
  def forward(self, x, hidden=None):
    """
    x의 shape (batch_size, seq_length, input_size)
    hidden이 반환하는 값은 (h_0, c_0)
    """
    batch_size, seq_length, _ = x.size()

    if hidden is None:
      # hidden state와 cell state를 0으로 초기화
      h_t = [x.new_zeros(batch_size, self.hidden_size) for _ in range(self.num_layers)]
      c_t = [x.new_zeros(batch_size, self.hidden_size) for _ in range(self.num_layers)]
    else:
      h_0, c_0 = hidden
      h_t = [h_0[i] for i in range(self.num_layers)]
      c_t = [c_0[i] for i in range(self.num_layers)]

    outputs = []

    # 시퀀스의 각 타임스텝에 대해 반복
    for t in range(seq_length):
      input_t = x[:, t, :]  # input_t의 shape : (batch_size, input_size)
      for layer_idx, cell in enumerate(self.layers): # cell은 말 그대로 해당 레이어의 LSTMCell 객체.
        h_prev = h_t[layer_idx]
        c_prev = c_t[layer_idx]
        h_next, c_next = cell(input_t, (h_prev, c_prev)) #
        h_t[layer_idx] = h_next  # 현재 레이어의 hidden state 업데이트
        c_t[layer_idx] = c_next  # 현재 레이어의 cell state 업데이트
        input_t = h_next  # 다음 레이어의 입력으로 사용
      outputs.append(h_next.unsqueeze(1))  # 출력 저장

    output = torch.cat(outputs, dim=1)  # output의 shape : (batch_size, seq_length, hidden_size)
    h_n = torch.stack(h_t)  # (num_layers, batch_size, hidden_size)
    c_n = torch.stack(c_t)  # (num_layers, batch_size, hidden_size)

    return output, (h_n, c_n)

## LSTM 실행(가상 데이터 사용)

In [10]:
import torch
import torch.nn as nn

# 모델 초기화
input_size = 10
hidden_size = 20
num_layers = 2
model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bias=True)

# 입력 데이터 생성
batch_size = 5
seq_length = 15
x = torch.randn(batch_size, seq_length, input_size)

# 초기 은닉 상태와 셀 상태 생성
h_0 = torch.zeros(num_layers, batch_size, hidden_size)
c_0 = torch.zeros(num_layers, batch_size, hidden_size)
hidden = (h_0, c_0)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 타겟 출력 생성
target = torch.randn(batch_size, seq_length, hidden_size)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output, (h_n, c_n) = model(x, hidden)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.9878
Epoch [2/10], Loss: 0.9871
Epoch [3/10], Loss: 0.9865
Epoch [4/10], Loss: 0.9859
Epoch [5/10], Loss: 0.9853
Epoch [6/10], Loss: 0.9847
Epoch [7/10], Loss: 0.9841
Epoch [8/10], Loss: 0.9835
Epoch [9/10], Loss: 0.9829
Epoch [10/10], Loss: 0.9823


In [5]:
# 데이터는 332X2임. feature는 가격과 날짜. 총 332일치의 나스닥지수 데이터

## 실제 데이터 사용